# Hotel Builder GA

You are the owner of a hotel company based in Los Angeles. You want to expand your business into San Francisco over the next 40 years. You know that it takes you 2 years to build a hotel and the city <u>does not allow you to build more than one hotel at a time</u>. 

Your Revenue in any year is given by the function $R(h)$ where $h$ is the number of hotels that are currently built (must be completed):

$$ R(h)=   \left\{
\begin{array}{ll}
      1500 + 100h - 5h^2 & h>0 \\
      0 & h = 0 \\
\end{array} 
\right.  $$

Each year your have to pay to run and maintain your completed hotels. The cost function $C(h)$ is:

$$ C(h) = 15 h$$

The cost to build a hotel is not fixed, it changes based on the year $y$ it's build is completed. The One-time Cost that is payed when you build a hotel and is represented by the function:

$$B(y) = 1000 \frac{y + 5}{2y} $$

<b>Use a genetic algorithm to determine the optimal building strategy for your company.</b>

In [ ]:
import random

time = 40
build_time = 2
def revenue(h):
  revenue = 1500 + 100*h - 5 *h**2
  return revenue if h > 0 else 0

def cost_to_maintain(h):
  return 15*h

def one_time_cost(y):
  return 1000 * (y + 5 / 2*y)

def random_knapsack_solution(time):
  build_prob = .5
  remaining_build_years = 0
  forcast = []
  for n in range(time):
      if remaining_build_years == 0 and n + build_time <= time:
          if random.random() < build_prob:
              remaining_build_years = build_time
          
      number_new_plants = 0
      if remaining_build_years == 1:
          number_new_plants = 1
          
      remaining_build_years = max(0, remaining_build_years - 1)
          
      forcast.append(number_new_plants)
  return forcast

print(random_knapsack_solution(time))

[0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]


In [ ]:
def knapsack_fitness(solution):
  count = 0
  one_time = []
  profit_list = []
  maintain = []
  for i in solution:
    if i == 1:
      count+=1
    one_time.append(one_time_cost(count))
    profit_list.append(revenue(count))
    maintain.append(cost_to_maintain(count))



  return sum(one_time),sum(profit_list) - sum(maintain)

sol = knapsack_fitness(random_knapsack_solution(time))

weight = sol[0]
value = sol[1]

print(f'the weight is: {weight} and the value is: {value}')



the weight is: 798000.0 and the value is: 69280


In [ ]:
POPULATION_SIZE = 50
def generate_knapsack_population(num_items, population_size):
    population = [random_knapsack_solution(num_items) for _ in range(population_size)]
    return population


In [ ]:
def roulette_selection(population):
  return random.sample(population,2)

def k_point_crossover(parent1,parent2,k):
  l =len(parent1)
  if k > l:
    print("Bad input")
    return -2
  indices = sorted(random.choices(range(l+1),k=k))
  print(indices)
  #flip it back and forth to decide which parent
  switch = 1
  child1 = []
  child2 = []
  for i,j in zip([0]+indices,indices+[l]):
    if switch:
      child1 += parent1[i:j]
      child2 += parent2[i:j]
    else:
      child1 += parent2[i:j]
      child2 += parent1[i:j]
    #xor with 1 to flip back and forth
    switch ^=1
  return child1,child2

def mutate_knapsack(solution,prob):
  return [s^1 if random.random() <prob else s for s in solution]


pop = generate_knapsack_population(time, POPULATION_SIZE)

parent1,parent2 = roulette_selection(pop)
child1,child2 = k_point_crossover(parent1,parent2,3)



[8, 13, 31]


In [ ]:
def generate_children(population,num_children,mutate_prob):
  children = []
  for _ in range(num_children//2):
    parent1,parent2 = roulette_selection(population)
    children += k_point_crossover(parent1,parent2,3)
  return [mutate_knapsack(child,mutate_prob) for child in children]

generate_children(generate_knapsack_population(time,50),50,.2)

In [ ]:
initial_population = generate_knapsack_population(time,POPULATION_SIZE)
NUM_CHILDREN = 50
MUTATE_PROB = 0.2


def tournament_selection(new_population,population_size,pop_fitness):
  trimmed_population = []
  for _ in range(population_size):
    (strat1,fit1),(strat2,fit2) = random.sample(list(zip(new_population,pop_fitness)),2)
    trimmed_population.append(strat1) if fit1>fit2 else trimmed_population.append(strat2)
  return trimmed_population

def ga_iteration(population, num_children,mutate_prob):
  children = generate_children(population,NUM_CHILDREN,MUTATE_PROB)
  new_population = population + children
  pop_fitness = [knapsack_fitness(solution) for solution in population]
  return tournament_selection(new_population,POPULATION_SIZE,pop_fitness)


NUM_ITER = 1000
current_population = initial_population
for _ in range(NUM_ITER):
  current_population = ga_iteration(current_population,NUM_CHILDREN,MUTATE_PROB)
pop_fitness = [knapsack_fitness(solution) for solution in current_population]
print(f"new max fitness: {max(pop_fitness)}")
NUM_ITER = 100
for _ in range(NUM_ITER):
  current_population = ga_iteration(current_population,NUM_CHILDREN,MUTATE_PROB)
pop_fitness = [knapsack_fitness(solution) for solution in current_population]
print(f"new max fitness: {max(pop_fitness)}")

Streaming output truncated to the last 5000 lines.
[22, 27, 33]
[5, 20, 29]
[1, 5, 24]
[5, 7, 30]
[2, 11, 30]
[11, 24, 29]
[11, 25, 32]
[0, 17, 25]
[3, 5, 17]
[7, 14, 20]
[8, 13, 32]
[14, 18, 20]
[18, 26, 27]
[2, 6, 31]
[1, 26, 32]
[5, 25, 32]
[5, 13, 28]
[6, 18, 27]
[2, 13, 17]
[1, 9, 35]
[6, 9, 21]
[7, 16, 26]
[13, 19, 35]
[5, 11, 39]
[2, 21, 28]
[7, 31, 33]
[3, 5, 18]
[7, 10, 22]
[10, 16, 39]
[4, 20, 29]
[7, 29, 38]
[13, 34, 37]
[25, 30, 34]
[15, 17, 27]
[19, 21, 34]
[19, 34, 40]
[16, 26, 40]
[24, 29, 36]
[19, 24, 24]
[28, 32, 33]
[9, 21, 37]
[1, 26, 37]
[9, 24, 39]
[9, 24, 33]
[8, 11, 24]
[5, 8, 30]
[9, 28, 35]
[25, 31, 40]
[5, 6, 27]
[2, 12, 27]
[10, 17, 34]
[11, 19, 40]
[6, 19, 26]
[3, 10, 27]
[18, 33, 38]
[9, 9, 25]
[16, 21, 37]
[1, 8, 38]
[5, 18, 21]
[11, 12, 28]
[5, 23, 35]
[3, 7, 15]
[1, 22, 36]
[0, 2, 12]
[3, 15, 34]
[3, 37, 39]
[24, 37, 38]
[1, 5, 22]
[7, 8, 30]
[1, 2, 34]
[6, 25, 38]
[13, 30, 33]
[13, 36, 39]
[8, 10, 33]
[21, 29, 40]
[8, 10, 11]
[6, 28, 35]
[8, 17, 30]
[3,